### OCI Data Science - Useful Tips
<details>
<summary><font size="2">Check for Public Internet Access</font></summary>

```python
import requests
response = requests.get("https://oracle.com")
assert response.status_code==200, "Internet connection failed"
```
</details>
<details>
<summary><font size="2">Helpful Documentation </font></summary>
<ul><li><a href="https://docs.cloud.oracle.com/en-us/iaas/data-science/using/data-science.htm">Data Science Service Documentation</a></li>
<li><a href="https://docs.cloud.oracle.com/iaas/tools/ads-sdk/latest/index.html">ADS documentation</a></li>
</ul>
</details>
<details>
<summary><font size="2">Typical Cell Imports and Settings for ADS</font></summary>

```python
%load_ext autoreload
%autoreload 2
%matplotlib inline

import warnings
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(format='%(levelname)s:%(message)s', level=logging.ERROR)

import ads
from ads.dataset.factory import DatasetFactory
from ads.automl.provider import OracleAutoMLProvider
from ads.automl.driver import AutoML
from ads.evaluations.evaluator import ADSEvaluator
from ads.common.data import ADSData
from ads.explanations.explainer import ADSExplainer
from ads.explanations.mlx_global_explainer import MLXGlobalExplainer
from ads.explanations.mlx_local_explainer import MLXLocalExplainer
from ads.catalog.model import ModelCatalog
from ads.common.model_artifact import ModelArtifact
```
</details>
<details>
<summary><font size="2">Useful Environment Variables</font></summary>

```python
import os
print(os.environ["NB_SESSION_COMPARTMENT_OCID"])
print(os.environ["PROJECT_OCID"])
print(os.environ["USER_OCID"])
print(os.environ["TENANCY_OCID"])
print(os.environ["NB_REGION"])
```
</details>

# Import Packages

In [1]:
from ads.jobs import Job, DataScienceJob, PythonRuntime
from ads import set_auth

set_auth("resource_principal")

# Creating the Job

In [ ]:
job = (
    Job(name="Training adult income with custom vars 3")
    .with_infrastructure(
        DataScienceJob()
        with_log_group_id('<your_log_group_ocid>')
        .with_shape_name("VM.Standard.E4.Flex")
        .with_shape_config_details(memory_in_gbs=4, ocpus=1)
    )
    .with_runtime(
        PythonRuntime()
        .with_service_conda("generalml_p311_cpu_x86_64_v1")
        .with_source("<job_folder_path>")
        .with_working_dir("<job_folder-name>") 
        .with_entrypoint("<entrypoint file name>") # no need full path. Only file name
        .with_output("output", "oci://<bucket_name>@<name_space>") # the output will be deleted after deprovisioning
        .with_environment_variable(TREES="PLACEHOLDER",DEPTH="PLACEHOLDER",LRATE="PLACEHOLDER") # these parameters are later customized in the job run using env_var
    )
)
job.create()

# Running the Job Run

In [ ]:
job_run = job.run(
    name="use job folder",
    env_var={'TREES': '50','DEPTH':'5', 'LRATE':'0.01'} # in every run we can change these values.
)
job_run.watch()